## Understand the dashboard

There are three important factors which should be understood: how county meetings are counted, how finished policy emc plans are counted, and how EMC workshop coverage is counted.

1) County meetings: the Report filters the _projects_ table on projects containing _county meeting_ in their name.
2) Finalized EMC plans: the Report uses only one clean table _counties-emc-plan-final-version-completed_. The SQL code of that table should be self explanatory.
3) Workshop coverage: the Report uses only one table _emc-workshopped-counties_. The SQL code of that table should be self explanatory.

## Establish format
A table needs to be created where each row is a county meetigns or EMC workshops.

Each of these rows can be used to send an API call that creates a project based on the county meeting and EMC workshop templates.

## Creating the data

Let's take the old data and rework it into a single table

In [1]:
# I have a custom module called dataload that helps me load all tables from a certain data folder
import pandas as pd
import os
import sys
sys.path.append(os.environ['pythonemkfdata'])
import dataload
import time

dataframes = dataload.load_emkf_data()

In [13]:
# The county_meetings table is already clean. I created it for the dummy dashboards
county_meetings = dataframes['county_meetings'].drop('Code', axis=1)

In [14]:
county_meetings['date'] = pd.to_datetime(county_meetings.date, dayfirst=True)

In [15]:
county_meetings.sample(5)

,county,date
45,Lamu,2019-08-22
91,Nakuru,2019-01-25
64,Tana River,2022-10-12
95,Mandera,2019-04-08
49,Vihiga,2022-04-05


Note that there are some county meetings already in Zoho Projects. We will only import the ones that are older than the month of May 2023

In [66]:
county_meetings = county_meetings.query('date < 20230501')

We now have a list of county meetings that can be imported in the API.

The old data for EMC plans is not very clear. There is a _Final EMC approval plan (YEAR)_ which is not helpful in our case. We need to coordinate with Programs and Wachira to understand what date to assign to the _final EMC plan_ task.

The rest of the exercise will show how to import old county meetings only.

## Injecting the data

### Initiating the API infrastructure

In [27]:
# You can find most of this code in the "Zoho API orchestration" notebook
# I will add more information in these lines for you to understand better what the code does.
import requests
import json
# get_close_matches function helps me match strings which are typed in slightly different manners
from difflib import get_close_matches

In [18]:
# The authentication information is taken from the self_client.json file.
# portals_info.json contains the Zoho Project portal ID needed for all operations that edit Projects
# refresh_tokens.json contains the refresh tokens that the API requires to create access tokens. See zoho OAUTH api documentation
# if this last sentence is not clear.
with open('./data/self_client.json') as client_info, open('./data/portals_info.json') as portals_info, open('./data/refresh_tokens.json') as tokens_file:
    client_app_data = json.load(client_info)
    zoho_portals_data = json.load(portals_info)
    tokens_dict = json.load(tokens_file)

In [21]:
TOKEN_URI = 'https://accounts.zoho.com/oauth/v2/token'
PROJECTS_API_URI = f'https://projectsapi.zoho.com/restapi/portal/{zoho_portals_data["projects_portal_id"]}/projects/'
# BOOKS_API_URI = 'https://www.zohoapis.com/books/v3'
CRM_APU_URI = 'https://www.zohoapis.com/crm/v4'

In [22]:
# Often, I work in different zoho Applications at the same time.
# Each application requires different requests headers. To make my life easier, I create a requests
# session for each application and store all of them in zoho_apps_api_sessions.
# For example, when I need to work with Zoho Books api, I will take the Zoho books session.
zoho_apps_api_sessions = tokens_dict['tokens'].copy()

refresh_payload = {
    'grant_type':'refresh_token',
    'client_id':client_app_data['client_id'],
    'client_secret':client_app_data['client_secret']
}

# All sessions need to be refreshed every now and then
for scope, refresh_token in tokens_dict['tokens'].items():
    refresh_payload['refresh_token'] = refresh_token
    refresh_response = requests.post(TOKEN_URI, data=refresh_payload)
    print(scope, refresh_response)
    
    # Create app specific session
    s = requests.session()
    s.headers = {
        'Authorization':f'Bearer {refresh_response.json()["access_token"]}'
    }
    # Store in sessions dict
    zoho_apps_api_sessions[scope] = s

portals_all <Response [200]>
projects_all <Response [200]>
crm_accounts_read <Response [200]>
crm_accounts_write <Response [200]>
projects_tasks_read <Response [200]>
projects_timesheets_read <Response [200]>
books_fullaccess_all <Response [200]>
books_expenses_all <Response [200]>
books_invoices_all <Response [200]>
books_bills_all <Response [200]>


### Working in the Zoho Projects API

In [28]:
# Using the requests session that contains auth headers for Zoho Projects
projects_session = zoho_apps_api_sessions['projects_all']

One hassle is to match each project with the correct CRM account. I have a CRM table from which I will draw the crm account IDs via string similarity search with the _get_close_matches_ function

In [2]:
crm_extract_counties = dataframes['crm_export'].query("`Account Type` == 'County'")[['Account ID', 'Account Name']]
crm_extract_counties.sample(3)

NameError: name 'dataframes' is not defined

In [43]:
crm_county_ids = crm_extract_counties.copy()
crm_county_ids.columns = ['crm_id', 'county']
crm_county_ids['crm_id'] = crm_county_ids.crm_id.str.split('_').str[1]
crm_county_ids['county'] = crm_county_ids.county.str.split(' County').str[0]
crm_county_ids = crm_county_ids.set_index('county')

crm_county_ids.sample(3)

,crm_id
county,
Nairobi,5621728000000484223
Migori,5621728000000485475
Kakamega,5621728000000485443


Let's take a look at how get_close_matches will help.

In [45]:
# Take the first county meeting
county_example = county_meetings.loc[0].county
# Find the closest matches
get_close_matches(county_example, crm_county_ids.index)

['Isiolo']

The result is not that amazing in this case. If the county was misspelt, it would still return the correct result. Take a look

In [47]:
get_close_matches('i so lo', crm_county_ids.index)

['Isiolo']

We need to add to each county meeting its matching crm account id

In [59]:
matching_county_names = county_meetings.county.apply(lambda x: get_close_matches(x, crm_county_ids.index)[0])
county_meetings['crm_id'] = crm_county_ids.loc[matching_county_names].reset_index().crm_id

In [61]:
county_meetings.sample(3)

,county,date,crm_id
11,Bomet,2023-02-16,5621728000000484827
1,Isiolo,2018-10-31,5621728000000485063
8,Elgeyo Marakwet,2018-06-11,5621728000000486095


We have all the information to populate the Zoho Project with County meetings information

To test if the implementation works, I will create a loop for each of the rows of the table but execute it only for the first row. To do this, I will add a break statement at the end of the loop. When I'm sure all the steps in the loop are executed properly, I will perform the full import

In [72]:
for i, row in county_meetings.iterrows():
    project_date = row.date.strftime('%m-%d-%Y')
    payload = {
        'name': row.county + ' County Meeting',
        'description':'This county meeting has been imported via API by Vittorio Rossi',
        'layout_id':'2072013000000020008', # Taken from this URL https://projects.zoho.com/portal/emkfoundation#setup/fieldsettings/2072013000000020008
        'group_id':'2072013000000312011', # The selected group is Advocacy
        'start_date':project_date,
        'end_date':project_date,
        'public':'yes',
        'UDF_CHAR1':'Advocacy',
        # The next line is updating the Account custom field
        'UDF_TEXT1':f'{{"module_id":"{row["crm_id"]}","value":"{row.county} County"}}'
    }
    create_project_response = projects_session.post(PROJECTS_API_URI, data=payload)
    try:
        proj_json = create_project_response.json()['projects'][0]
        proj_id = proj_json['id']
        proj_name = proj_json['name']
        print(i, "Created proj successfully", proj_name)
        
        update_project_via_id_uri = PROJECTS_API_URI + str(proj_id) + '/'
        update_proj_response = projects_session.post(update_project_via_id_uri, data={'custom_status':'2072013000000020116'})
        time.sleep(.3)
    except KeyError:
        print('Failed to creat proj')
    # break
    # the break statement is now commented out for the full import

0 Created proj successfully Isiolo County Meeting
1 Created proj successfully Isiolo County Meeting
2 Created proj successfully Marsabit County Meeting
3 Created proj successfully Uasin Gishu County Meeting
4 Created proj successfully Uasin Gishu County Meeting
5 Created proj successfully Uasin Gishu County Meeting
6 Created proj successfully Elgeyo Marakwet County Meeting
7 Created proj successfully Elgeyo Marakwet County Meeting
8 Created proj successfully Elgeyo Marakwet County Meeting
9 Created proj successfully Kericho County Meeting
10 Created proj successfully Kericho County Meeting
11 Created proj successfully Bomet County Meeting
12 Created proj successfully Bomet County Meeting
13 Created proj successfully Nandi County Meeting
14 Created proj successfully Tharaka Nithi County Meeting
15 Created proj successfully Tharaka Nithi County Meeting
16 Created proj successfully Kwale County Meeting
17 Created proj successfully Kwale County Meeting
18 Created proj successfully Kwale Co